In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import sklearn
from sklearn.metrics import precision_recall_fscore_support as pr
from sklearn.metrics import average_precision_score, precision_recall_curve
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score, KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_curve, auc,roc_auc_score
from keras.optimizers import SGD, Adam, RMSprop
import scipy
from keras.layers.normalization import BatchNormalization
from keras.callbacks import TensorBoard
import datetime
from random import shuffle
import json
import matplotlib.pyplot as plt
from keras import regularizers
import tensorflow as tf
from keras import backend as K
import scipy

/home/ubuntu/anaconda3/envs/tensorflow_p27/lib/python2.7/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p27/lib/python2.7/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p27/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
from tensorflow.python.client import device_lib
print device_lib.list_local_devices()

In [2]:
%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0,4.0)

# Load Data

In [2]:
import numpy as np
import pandas as pd
data_dir = '/home/ubuntu/cs230/FeedForward/Data/XY_data/'
data_dir2 = '/home/ubuntu/cs230/FeedForward/Data/XY_data_trimmed/'

print('Getting Training X Data')
x_train = pd.read_table(data_dir2 + 'x_train_trimmed.txt',sep = '\t')
print('Getting Training Y Data')
y_train = pd.read_table(data_dir + 'y_train.txt', sep = '\t')
print('Getting Dev X Data')
x_dev = pd.read_table(data_dir2 + 'x_dev_trimmed.txt',sep = '\t')
print('Getting Dev Y Data')
y_dev = pd.read_table(data_dir + 'y_dev.txt',sep = '\t')
print('Getting Test X Data')
x_test = pd.read_table(data_dir2 + 'x_test_trimmed.txt', sep = '\t')
print('Getting Test Y Data')
y_test = pd.read_table(data_dir + 'y_test.txt', sep = '\t')


Getting Training X Data
Getting Training Y Data
Getting Dev X Data
Getting Dev Y Data
Getting Test X Data
Getting Test Y Data


In [3]:
# drop last rows (due to error upon original download)
x_dev.drop(x_dev.index[len(x_dev)-1], inplace = True)
x_test.drop(x_test.index[len(x_test)-1], inplace = True)
y_test.drop(y_test.index[len(y_test)-1], inplace = True)
y_dev.drop(y_dev.index[len(y_dev)-1], inplace = True)

# Prep Data

In [4]:
y_train = scipy.sign(y_train)
y_dev = scipy.sign(y_dev)
y_test = scipy.sign(y_test)

In [5]:
# Due to known Keras issue #8011
def prepareForWeights(mat):
    width = mat.shape[1] + 2
    length = mat.shape[0]
    z = np.zeros((length, width))
    z[:,2:] = mat
    z[:,1] = 1
    z[:,0] = 0
    return z

In [6]:
y_train = prepareForWeights(y_train)
y_test = prepareForWeights(y_test)
y_dev = prepareForWeights(y_dev)

# Metrics and Loss Functions

In [7]:
def flattenMatrix(mat):
    mat = pd.DataFrame(mat)
    n = len(mat)
    flattened = []
    for i in range(n):
        actual_vals = mat.loc[i, ].tolist()
        flattened.extend(actual_vals)
    return flattened

def calculateF1Score(preds, test):
    actual = flattenMatrix(test)
    predicted = flattenMatrix(preds)
    actual_binary = [1 if x > 0 else 0 for x in actual]
    predicted_binary = [1 if x > 0 else 0 for x in predicted]
    bPrecis, bRecall, bFscore, bSupport = pr(actual_binary, predicted_binary, average='binary')
    return bPrecis, bRecall, bFscore

In [8]:
class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.val_losses = []
    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))


In [9]:
def weightedLoss(w):
    def loss(y_true, y_pred):
        y_true_edited = y_true[:,2:]
        y_pred_edited = y_pred[:,2:]
        l = tf.reduce_mean(tf.nn.weighted_cross_entropy_with_logits(targets=y_true_edited,logits=y_pred_edited,pos_weight=w))
        return l
    return loss

def weighted_binary_crossentropy(zero_weight, one_weight):

    def weighted_binary_crossentropy(y_true, y_pred):

        # Calculate the binary crossentropy
        b_ce = K.binary_crossentropy(y_true, y_pred)

        # Apply the weights
        weight_vector = y_true * one_weight + (1. - y_true) * zero_weight
        weighted_b_ce = weight_vector * b_ce

        # Return the mean error
        return K.mean(weighted_b_ce)

    return weighted_binary_crossentropy

def binary_accuracy_V2(y_true, y_pred):
    '''Calculates the mean accuracy rate across all predictions for binary
    classification problems.
    '''
    y_true_edited = y_true[:,2:]
    y_pred_edited = y_pred[:,2:]
    return K.mean(K.equal(y_true, K.round(y_pred)))

In [11]:
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall))


In [12]:
def flattenMatrix(mat):
    mat = pd.DataFrame(mat)
    n = len(mat)
    flattened = []
    for i in range(n):
        actual_vals = mat.loc[i, ].tolist()
        flattened.extend(actual_vals)
    return flattened

def calculateF1ScoreOnly(y_true, y_pred):
    actual_binary = [1 if x > 0 else 0 for x in y_true]
    predicted_binary = [1 if x > 0 else 0 for x in y_pred]
    bPrecis, bRecall, bFscore, bSupport = pr(actual_binary, predicted_binary, average='binary')
    return bFscore

def calculateF1Score(y_true, y_pred):
    actual = flattenMatrix(y_true)
    predicted = flattenMatrix(y_pred)
    y_true2 = np.reshape(y_true, (y_true.shape[0]*y_true.shape[1],-1))
    y_pred2 = np.reshape(y_pred, (y_pred.shape[0]*y_pred.shape[1],-1))
    bPrecis, bRecall, bFscore, bSupport = pr(y_true2, y_pred2, average='binary')
    return bPrecis, bRecall, bFscore

# Hyperparameter Searching

In [13]:
import math
# if scale == None: generates uniform random value between start/end
# if scale == 'log': generate random variable r in [log(start),log(end)], then return 10^r
#     ex. if you input start:0.0001, end:1 it will return 10^r, where r in [-4,0]
def random_search(start, end, scale='uniform'):
    if scale == 'uniform':
        return np.random.uniform(start, end)
    elif scale == 'int':
        return np.random.randint(start,end)
    elif scale == 'log':
        a = math.log(start, 10)
        b = math.log(end, 10)
        r = np.random.uniform(a, b)
        return 10**r
    else:
        return 'ERROR'

# get random value from list
def random_grid_search(vals):
    length = len(vals)
    return vals[np.random.randint(0,length)]

def decrement_num_neurons(first, min_val, num_layers):
    layers = [first]
    random = np.random.rand()
    if random > 0.5:
        random = 50
    else:
        random = 0
    prev = first
    for j in range(num_layers-1):
        prev = max(prev-25, min_val) 
        layers.append(prev)
    return layers

def dropout_search(num_layers):
    possible_vals = [0.4,0.35,0.3, 0.25, 0.2, 0.15, 0.1, 0.05, 0]
    d = []
    prev = None
    for l in range(num_layers-1):
        if l == 0:
            dl = possible_vals[np.random.randint(0,len(possible_vals))]
            d.append(dl)
            prev = dl
        else:
            dl = possible_vals[np.random.randint(0,len(possible_vals))]
            while dl > prev:
                dl = possible_vals[np.random.randint(0,len(possible_vals))]
            d.append(dl)
            prev = dl
    d.append(0)
    return d

# Neural Network

In [14]:
'''
Creates NN
'''
def neural_network(num_layers, num_neurons, learning_rate, activation, dropout_rate, x_shape, y_shape, pw, lam):
    assert len(num_neurons) == len(dropout_rate)
    K.clear_session()
    model = Sequential()
    model.add(Dense(num_neurons[0], activation=activation, input_dim=x_shape[1]))
    for l in range(1, num_layers-2):
        model.add(Dense(num_neurons[l], kernel_regularizer = regularizers.l2(lam)))
        model.add(BatchNormalization())
        model.add(Activation(activation))
        model.add(Dropout(dropout_rate[l]))
    model.add(Dense(y_shape[1], activation='sigmoid'))
    adam = Adam(lr=learning_rate)
    model.compile(loss=weighted_binary_crossentropy(1,pw),
              optimizer=adam,
              metrics = [binary_accuracy_V2, f1])
    return model

    


In [15]:
def run_nn(x_train, y_train, x_dev, y_dev, params, now):
    num_layers = params['num_layers']
    num_neurons = params['num_neurons']
    learning_rate = params['learning_rate']
    activation = params['activation']
    dropout_rate = params['dropout_rate']
    epochs = params['epochs']
    pw = params['weight']
    lam = params['lambda']
    
    x_shape = x_train.shape
    y_shape = y_train.shape

    os.makedirs('log_dir_keras/{}'.format(now))
    tensorboard = TensorBoard(log_dir="log_dir_keras/{}".format(now))
    losshistory = LossHistory()
    print("Generating NN Architecture")
    model = neural_network(num_layers, num_neurons, learning_rate, activation, dropout_rate, x_shape, y_shape, pw, lam)
    print('Fitting Model')
    mod_hist = model.fit(x_train, y_train, validation_data = (x_dev , y_dev), verbose=1, batch_size = 512, epochs = epochs, callbacks = [losshistory])

    preds = model.predict(x_dev)
    preds = preds[:,2:]
    preds[preds >= 0.5] = 1
    preds[preds < 0.5] = 0
    bPrecis, bRecall, bFscore = calculateF1Score(y_dev[:,2:], preds)
    print('Precision: {} , Recall : {} , F1 : {} '.format(bPrecis, bRecall, bFscore))

    return bFscore, model, mod_hist.history

In [16]:
def initialize_keys(d,now):
    keys = ['parameters','loss','val_loss','f1','val_f1', 'acc', 'val_acc', 'test_fscore', 'test_precision', 'test_recall']
    new_entry = {}
    for k in keys:
        new_entry[k] = []
        
    d[now] = new_entry
    
    return d

'''
{'f1': [0.12090024837345423, 0.1811971694957791], 
'binary_accuracy_V2': [0.79680669177864871, 0.91634968441619902], 
'loss': [0.95547696070291122, 0.84549003041767801], 
'val_binary_accuracy_V2': [0.97204344202490411, 0.97537883239633894], 
'val_f1': [nan, nan], 
'val_loss': [0.79929518769769103, 0.78799666867536655]}
'''
def update_dicts(d, now, params, mod_hist):
    
    d[now]['parameters'] = params

    d[now]['loss'] = mod_hist['loss']
    d[now]['val_loss'] = mod_hist['val_loss']
    d[now]['f1'] = mod_hist['f1']
    d[now]['val_f1'] = mod_hist['val_f1']
    d[now]['acc'] = mod_hist['binary_accuracy_V2']
    d[now]['val_acc'] = mod_hist['val_binary_accuracy_V2']
    return d

In [ ]:
# Search ranges were refined using a coarse-to-fine method
K.clear_session()
pw_low = 5
pw_high = 15
learning_rate_low = 0.001
learning_rate_high = 0.01
num_layers_low = 6
num_layers_high = 13
num_neurons_1 = [200,250,300,400,500]
dropout_low = 0
dropout_high = 0.2
num_epochs = 3

f_score_max = -100
best_model = None
model_losses = []
model_val_losses = []
model_dicts = {}

model_time_start = str(datetime.datetime.now())
for i in range(4):
    K.clear_session()
    now = str(datetime.datetime.now())
    model_dicts = initialize_keys(model_dicts,now)

    iteration_param_dict = {}
    pw = random_search(pw_low,pw_high, scale='int') # positive error weight 
    learning_rate = random_search(learning_rate_low,learning_rate_high,scale='log')
    num_layers = random_search(num_layers_low,num_layers_high, scale='int')
    num_neurons = decrement_num_neurons(random_grid_search(num_neurons_1), 50, num_layers)
    dropout = [random_search(dropout_low, dropout_high, 'uniform') for i in range(len(num_neurons))]
    dropout = dropout_search(num_layers)
    lam = float(np.random.randint(1,5))/1000

    iteration_param_dict['weight'] = pw
    iteration_param_dict['learning_rate'] = learning_rate
    iteration_param_dict['num_layers'] = num_layers
    iteration_param_dict['num_neurons'] = num_neurons
    iteration_param_dict['activation'] = 'relu'
    iteration_param_dict['dropout_rate'] = dropout
    iteration_param_dict['epochs'] = num_epochs
    iteration_param_dict['lambda'] = lam
    print('########')
    print(iteration_param_dict)

    f_score, model_i, mod_hist = run_nn(x_train, y_train ,x_dev, y_dev, iteration_param_dict, now)

    model_dicts = update_dicts(model_dicts, now, iteration_param_dict, mod_hist)
    with open('model_dictionaries/model_dictionary_{}.json'.format(model_time_start), 'w') as mdj:
        json.dump(model_dicts, mdj)

    if f_score > f_score_max:
        f_score_max = f_score
        model_i.save('best_models/Feed_Forward_Best_Model_{}.h5'.format(model_time_start))
        best_model = model_i

# Graph Functions

In [17]:
def plot_roc_curve(y_preds, y_true,title,text):
    '''
    Plots a ROC curve for given set of predictions
    
    parameters:
        @predict_results: predict generator object
    return:
        None
    '''
    
    (dim1,dim2) = y_preds.shape
    
    y = np.asarray(y_true).reshape((dim1*dim2,1))
    y_hat = np.asarray(y_preds).reshape((dim1*dim2,1))
    
    fpr, tpr, _ = roc_curve(y,y_hat)

    AUROC = auc(fpr,tpr)
    plt.plot(fpr,tpr,label=text + ' (AUC = %.3f)' % AUROC)
    plt.legend(loc='lower right')
    plt.title(title)
    plt.xlabel('1 - Specificity')
    plt.ylabel('Sensitivity')
    plt.show()
    
    return

In [18]:
def plot_roc_curve_2_sets(y_preds1, y_true1, y_preds2, y_true2, title,text1, text2):
    '''
    Plots a ROC curve for given set of predictions
    
    parameters:
        @predict_results: predict generator object
    return:
        None
    '''
    
    (dim1,dim2) = y_preds1.shape
    
    y = np.asarray(y_true1).reshape((dim1*dim2,1))
    y_hat = np.asarray(y_preds1).reshape((dim1*dim2,1))
    
    fpr, tpr, _ = roc_curve(y,y_hat)

    AUROC = auc(fpr,tpr)
    plt.plot(fpr,tpr,label=text1 + ' AUC = %.3f' % AUROC)
    

    y2 = y_true2
    y_hat2 = y_preds2
    
    fpr2, tpr2, _ = roc_curve(y2,y_hat2)
    AUROC2 = auc(fpr2,tpr2)
    plt.plot(fpr2,tpr2,label=text2 + ' AUC = %.3f' % AUROC2)
    
    plt.legend(loc='lower right')
    plt.title(title)
    plt.xlabel('1 - Specificity')
    plt.ylabel('Sensitivity')
    plt.savefig('roc_curve.png')
    plt.show()    
    return

In [19]:
def plot_pr_curve(y_preds,y_true,title,text):
    '''
    Plots a PR Curve for a given set of predictions
     
    parameters:
    
    return:
        None
    '''    

    (dim1,dim2) = y_preds.shape
    
    y = np.asarray(y_true).reshape((dim1*dim2,1))
    y_hat = np.asarray(y_preds).reshape((dim1*dim2,1))

    precision, recall, thresholds = precision_recall_curve(y, y_hat)
    AUPR = average_precision_score(y,y_hat)
    
    plt.plot(recall,precision,label=text + ' (AP = %.3f)' % AUPR,color='r')
    plt.legend(loc='upper right')
    plt.title(title)
    plt.xlabel('Recall (p(y_hat ==1 | y==1))')
    plt.ylabel('Precision')
    plt.show()
    
    return

In [20]:
def plot_pr_curve_2_sets(y_preds,y_true, y_preds2, y_true2, title,text1,text2):
    '''
    Plots a PR Curve for a given set of predictions
     
    parameters:
    
    return:
        None
    '''    

    (dim1,dim2) = y_preds.shape
    
    y = np.asarray(y_true).reshape((dim1*dim2,1))
    y_hat = np.asarray(y_preds).reshape((dim1*dim2,1))

    precision1, recall1, thresholds1 = precision_recall_curve(y, y_hat)
    area = auc(recall1, precision1)
    AUPR = average_precision_score(y,y_hat)
    
    plt.plot(recall1,precision1,label=text1 + ' AUC = %.3f' % area)
    
    y2 = y_true2
    y_hat2 = y_preds2

    precision2, recall2, thresholds2 = precision_recall_curve(y2, y_hat2)
    area2 = auc(recall2, precision2)
    AUPR2 = average_precision_score(y2,y_hat2)
    
    plt.plot(recall2,precision2,label=text2 + ' AUC = %.3f' % area2)
    
    plt.legend(loc='upper right')
    plt.title(title)
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.savefig('pr_curve.png')
    plt.show()
    return

# Compare to Human Authored 

In [ ]:
# Example Models:
# best_models/Feed_Forward_Best_Model_2018-03-19 02:22:23.891330.h5
# best_models/Feed_Forward_Best_Model_2018-03-19 08:17:43.787454.h5

In [21]:
'''
from keras.utils import CustomObjectScope

with CustomObjectScope({'weighted_binary_crossentropy': weighted_binary_crossentropy(1,10),
                        'binary_accuracy_V2' : binary_accuracy_V2,
                       'f1':f1}):
    model = keras.models.load_model('best_models/Feed_Forward_Best_Model_2018-03-19 02:22:23.891330.h5')
'''
from keras.utils import CustomObjectScope

with CustomObjectScope({'weighted_binary_crossentropy': weighted_binary_crossentropy(1,10),
                        'binary_accuracy_V2' : binary_accuracy_V2,
                       'f1':f1}):
    model = keras.models.load_model('best_models/Feed_Forward_Best_Model_2018-03-19 08:17:43.787454.h5')

In [22]:
test_preds = model.predict(x_test)

In [ ]:
ha_dir = '/home/ubuntu/cs230/HumanBaseline/'
human_true = pd.read_table(ha_dir + 'human_response.txt', header = None)
human_true.columns = ['human_true']

human_pred = pd.read_table(ha_dir + 'human_prediction.txt', header = None)
human_pred.columns = ['human_pred']


In [ ]:
plot_roc_curve_2_sets(test_preds[:,2:],y_test[:,2:], human_pred['human_pred'], human_true['human_true'], 'ROC Curve - ClinicNet vs Order Sets','ClinicNet', 'Order Sets')

In [ ]:
plot_pr_curve_2_sets(test_preds[:,2:],y_test[:,2:], human_pred['human_pred'], human_true['human_true'], 'PR Curve - ClinicNet vs Order Sets','ClinicNet', 'Order Sets')

# AUROC Comparison

In [ ]:
from keras.utils import CustomObjectScope

with CustomObjectScope({'weighted_binary_crossentropy': weighted_binary_crossentropy(1,10),
                        'binary_accuracy_V2' : binary_accuracy_V2,
                       'f1':f1}):
    model = keras.models.load_model('best_models/Feed_Forward_Best_Model_2018-03-19 08:17:43.787454.h5')

test_preds = model.predict(x_test)

data_dir = '/home/ubuntu/cs230/FeedForward/Data/XY_data/'
y_train_to_get_column_names = pd.read_table(data_dir + 'y_train.txt', sep = '\t',nrows = 3)


test_preds_touse = test_preds[:,2:]
y_test_touse = y_test[:,2:]
print(test_preds_touse.shape)
print(y_test_touse.shape)

'''
Model AUROC
'''
print test_preds.shape

response_names = y_train_to_get_column_names.columns
results = pd.DataFrame(columns = ['Clinical_Item_Net', 'AUROC_Net', 'Precision_Net', 'Recall_Net', 'F1_Net'])

for response_num in range(test_preds_touse.shape[1]):
    yi_pred = test_preds_touse[:,response_num]
    yi_actual = y_test_touse[:,response_num]
    
    yi_pred_cutoff = yi_pred
    yi_pred_cutoff[yi_pred_cutoff >=0.5] = 1
    yi_pred_cutoff[yi_pred_cutoff <0.5] = 0
    
    bPrecis_, bRecall_, bFscore_, _ = pr(yi_actual, yi_pred_cutoff , average='binary')
    #fpr, tpr, _ = roc_curve(yi_pred, yi_actual)
    try:
        AUROC = roc_auc_score(yi_actual,yi_pred)
    except:
        print(response_num)
        AUROC = None
    
    results.loc[response_num, 'Clinical_Item_Net'] = response_names[response_num]
    results.loc[response_num, 'AUROC_Net'] = AUROC 
    results.loc[response_num, 'Precision_Net'] = bPrecis_ 
    results.loc[response_num, 'Recall_Net'] = bRecall_ 
    results.loc[response_num, 'F1_Net'] = bFscore_

'''
HA AUROC
'''
ha_metrics = pd.read_table('/home/ubuntu/cs230/HumanBaseline/results_for_humanauthored.txt', sep = '\t')
merged = results.merge(ha_metrics, left_on='Clinical_Item_Net', right_on='Clinical_Item', how='inner')

merged['AUROC_Diff'] = merged['AUROC_Net'] - merged['AUROC']
merged['Precision_Diff'] = merged['Precision_Net'] - merged['precision']
merged['Recall_Diff'] = merged['Recall_Net'] - merged['recall']
merged['F1_Diff'] = merged['F1_Net'] - merged['F1_Score']

merged = merged.dropna() # drop rows with only 1 class in y_true, resulting in errors when calculating ROC

fig,ax = plt.subplots(1)
norm = MidpointNormalize(midpoint=0)
plt.scatter(merged['Clinical_Item'], merged['AUROC_Diff'],s=2,c=merged['AUROC_Diff'], cmap='RdBu')
plt.axhline(y=0, color='black', linestyle='-')

plt.ylim(-.6, .6) 
ax.set_xticklabels([])
plt.xticks([])
plt.title('AUROC by Clinical Item Comparison')
plt.xlabel('Clinical Item Number')
plt.ylabel('AUROC Difference (ClinicNet - Order Sets)')
plt.savefig('auroc_comparison.png')
plt.show()
#plt.savefig()


In [ ]:
x = list(merged['AUROC_Diff'])
plt.hist(x, bins = 25)
plt.axvline(statistics.median(x), color='black', linestyle='dashed', linewidth=2, label = 'Median')
plt.axvline(0, color='black', linewidth=2, label = '0')

plt.title('AUROC by Clinical Item Comparison')
plt.xlabel('AUROC (ClinicNet) - AUROC (Order Sets)')
plt.ylabel('Count')
plt.legend(loc='upper right')
plt.savefig('HB_comparison.png')